In [36]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import matplotlib.cm as cm


def readifrit(path, nvar=0, moden=2, skipmoden=2):
    import struct
    import numpy as np
    openfile=open(path, "rb")
    dump = np.fromfile(openfile, dtype='i', count=moden)
    N1,N2,N3=np.fromfile(openfile, dtype='i', count=3)
#     print(N1,N2,N3)
    dump = np.fromfile(openfile, dtype='i', count=moden)
    data = np.zeros([N1, N2, N3])
    j = 0
    for i in range(nvar):
        openfile.seek(4*skipmoden, 1)
        for j in range(4):
            openfile.seek(N1*N2*N3, 1)
        openfile.seek(4*moden, 1)
    openfile.seek(4*skipmoden, 1)
    data[:, :, :] = np.reshape(np.fromfile(openfile, dtype='f4', count=N1 * N2 * N3), [N1, N2, N3])
    openfile.close()
    return N1,N2,N3,data


def downsample(data,f=2):
    data_new=(data[::2, ::2, ::2]+data[1::2, ::2, ::2]+
              data[::2, 1::2, ::2]+data[::2, ::2, 1::2]+
              data[1::2, 1::2, ::2]+data[::2, 1::2, 1::2]+
              data[1::2, ::2, 1::2]+data[1::2, 1::2, 1::2])
    return data_new

def pk(data, box_size, k_list_phys, mode=0, usefftw=False):
    N = data.shape[0]
    k_list = k_list_phys*box_size/N
    data=np.fft.rfftn(data)
    kx, ky, kz = np.mgrid[:N, :N, :(N/2+1)]
    kx[kx > N/2-1] = kx[kx > N/2-1]-N
    ky[ky > N/2-1] = ky[ky > N/2-1]-N
    kz[kz > N/2-1] = kz[kz > N/2-1]-N
    k=2.0*np.pi*np.sqrt(kx**2+ky**2+kz**2)/N
    if mode == 1:
        kf = 2.0*np.pi/N
        res = np.zeros(len(k_list)-1)
        for i in range(len(k_list)-1):
            if np.sum((k >= k_list[i]) & (k < k_list[i+1]))>0:
                res[i] = np.mean(np.abs(data[(k >= k_list[i]) & (k < k_list[i+1])])**2)
        return res*box_size**3 / N**6
    if mode==0:
        kf=2.0*np.pi/N
        h1, dump = np.histogram(k.flat,weights=np.abs(data.flat)**2,bins=k_list)
        h2, dump = np.histogram(k.flat,bins=k_list)
        h2[h2==0] = 1.0
        res = h1/h2
        return res*box_size**3/N**6
    if mode==2:
        kf=2.0*np.pi/N
        res=np.zeros(len(k_list))
        for i in range(len(k_list)):
            res[i]=np.mean(np.abs(data[(k>=k_list[i]-kf) & (k<k_list[i]+kf)])**2)
        return res*box_size**3/N**6
    if mode==2:
        return k,data

def pk2(data, data2, box_size, k_list_phys, mode=0, usefftw=False):
    N = data.shape[0]
    k_list = k_list_phys*box_size/N
    data=np.fft.rfftn(data)
    data2=np.fft.rfftn(data2)
    kx, ky, kz = np.mgrid[:N, :N, :(N/2+1)]
    kx[kx > N/2-1] = kx[kx > N/2-1]-N
    ky[ky > N/2-1] = ky[ky > N/2-1]-N
    kz[kz > N/2-1] = kz[kz > N/2-1]-N
    k=2.0*np.pi*np.sqrt(kx**2+ky**2+kz**2)/N
    if mode == 1:
        kf = 2.0*np.pi/N
        res = np.zeros(len(k_list)-1)
        for i in range(len(k_list)-1):
            if np.sum((k >= k_list[i]) & (k < k_list[i+1]))>0:
                res[i] = np.mean((data[(k >= k_list[i]) & (k < k_list[i+1])]
                                      * np.conjugate(data2[(k >= k_list[i]) & (k < k_list[i+1])])))
        return res*box_size**3 / N**6
    if mode==0:
        kf=2.0*np.pi/N
        h1, dump = np.histogram(k.flat,weights=np.real(data.flatten()*np.conjugate(data2.flatten())),bins=k_list)
        h2, dump = np.histogram(k.flat,bins=k_list)
        h2[h2==0] = 1.0
        res = h1/h2
        return res*box_size**3/N**6
    if mode==2:
        kf=2.0*np.pi/N
        res=np.zeros(len(k_list))
        for i in range(len(k_list)):
            res[i]=np.mean(np.abs(data[(k>=k_list[i]-kf) & (k<k_list[i]+kf)])**2)
        return res*box_size**3/N**6
    if mode==3:
        kf = 2.0*np.pi/N
        res = np.zeros(len(k_list)-1)
        res2 = np.zeros(len(k_list)-1)
        for i in range(len(k_list)-1):
            if np.sum((k >= k_list[i]) & (k < k_list[i+1]))>0:
#                 print(np.corrcoef(x=data[(k >= k_list[i]) & (k < k_list[i+1])].flatten(),
#                                       y=data2[(k >= k_list[i]) & (k < k_list[i+1])].flatten())[0][1])
                res[i] = (np.corrcoef(x=data[(k >= k_list[i]) & (k < k_list[i+1])].flatten(),
                                      y=data2[(k >= k_list[i]) & (k < k_list[i+1])].flatten())[0][1])
                res2[i] = np.mean(data[(k >= k_list[i]) & (k < k_list[i+1])].flatten() * 
                                  np.conjugate(data2[(k >= k_list[i]) & (k < k_list[i+1])].flatten()))
        return res, res2

#####

machine = 'ias'
%matplotlib inline

i_list=[0,1,2,3,4,5,6,7]

In [37]:
# T10 = 0.068 # K   */
# A10 = 2.85e-15 # 1/s */

# Tcmb = 2.728/auni[min_level];
# xa = T10/(A10*Tcmb)*nxi_lya*3e10*4.5e-18*4/27.0;

In [38]:
files1 = glob.glob('/home/kaurov/scratch/rei/shuffle10/xxxOUT_xshuffle_%04i_%04i/*.bin'%(0,0))
files2 = glob.glob('/home/kaurov/scratch/rei/shuffle10/xxxOUT_xshuffle_%04i_%04i/*.bin'%(64,7))

files1.sort()
files2.sort()

snaps_n1 = np.array([np.float(files1[i][-10:-4]) for i in range(len(files1))])
snaps_n2 = np.array([np.float(files2[i][-10:-4]) for i in range(len(files2))])

print(snaps_n1)
print(snaps_n2)

[ 0.0109  0.0112  0.0326  0.0336  0.0346  0.0354  0.0363  0.0372  0.0381
  0.0391  0.04    0.0409  0.0417  0.0426  0.0435  0.0454  0.0464  0.0474
  0.0482  0.05    0.0509  0.0518  0.0528  0.0548  0.0556  0.0582  0.0591
  0.062   0.063   0.064   0.0648  0.0665  0.0674  0.0683  0.0692  0.0711
  0.0721  0.0731  0.0741  0.0766  0.0775  0.0793  0.0802  0.083   0.084
  0.086   0.087   0.0904  0.0913  0.0931  0.0941  0.0959  0.0969  0.0999
  0.1009  0.1028  0.1036  0.1063  0.1072]
[ 0.0109  0.0112  0.0115  0.0118  0.0121  0.0125  0.0128  0.0132  0.0136
  0.014   0.0144  0.0326  0.0336  0.0346  0.0354  0.0363  0.0372  0.0381
  0.0391  0.04    0.0409  0.0417  0.0426  0.0435  0.0454  0.0464  0.0474
  0.0482  0.05    0.0509  0.0518  0.0528  0.0548  0.0556  0.0582  0.0591
  0.062   0.063   0.064   0.0648  0.0665  0.0674  0.0683  0.0692  0.0711
  0.0721  0.0731  0.0741  0.0766  0.0775  0.0793  0.0802  0.083   0.084
  0.086   0.087   0.0904  0.0913  0.0931  0.0941  0.0959  0.0969  0.0999
  0.1009  0

In [43]:
a = 0.11
print(1/a-1)
i1 = np.where(snaps_n1>=a)[0][0]
i2 = np.where(snaps_n2>=a)[0][0]

print(snaps_n1[i1], snaps_n2[i2])

temp1 = readifrit(files1[i1], nvar=3)[3]
temp2 = readifrit(files2[i1], nvar=3)[3]
temp121 = readifrit(files1[i1], nvar=2)[3]
temp221 = readifrit(files2[i1], nvar=2)[3]

T10 = 0.068 # K   */
A10 = 2.85e-15 # 1/s */

Tcmb = 2.728/a
xa_mult = T10/(A10*Tcmb)*3e10*4.5e-18*4/27.0


from matplotlib.colors import LogNorm
plt.figure(figsize=(6,6), dpi=300)
plt.subplot(221)
plt.imshow(np.mean(temp1[:,:,50:51]*xa_mult,2), norm=LogNorm(vmin=0.01, vmax=1))
plt.colorbar()
plt.subplot(222)
plt.imshow(np.mean(temp2[:,:,50:51]*xa_mult,2), norm=LogNorm(vmin=0.01, vmax=1))
plt.colorbar()
plt.subplot(223)
plt.imshow((temp121[:,:,50]), cmap='bwr', vmin=-0.005, vmax=0.005)
plt.colorbar()
plt.subplot(224)
plt.imshow((temp221[:,:,50]), cmap='bwr', vmin=-0.005, vmax=0.005)
plt.colorbar()

8.090909090909092


IndexError: index 0 is out of bounds for axis 0 with size 0